In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow  as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

from tensorflow.keras.callbacks import *

from sklearn.metrics import mean_absolute_error
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

import os

In [4]:
frame = pd.read_csv(r'ausleihe.csv')
df = pd.DataFrame(frame)
col = list(df.columns.values)

df.head()

,Year,Week,LAT,LONG,count
0,2015,1,40.680342,-73.955769,18
1,2015,1,40.680983,-73.950048,18
2,2015,1,40.682166,-73.953990,8
3,2015,1,40.682232,-73.961458,19
4,2015,1,40.683125,-73.978951,59


In [5]:
#get input and output
x = df.drop("count",axis=1)
y = df["count"]

In [6]:
#split data into train and test
x_train, x_test ,y_train , y_test = train_test_split(x,y,test_size=0.2,shuffle=False)

In [11]:
#normalise data
ct = make_column_transformer(
 (MinMaxScaler(),["Year","Week","LAT","LONG"]) , remainder="passthrough"
 )

ct.fit(x_train)

x_train_normalized = ct.transform(x_train)

x_test_normalized = ct.transform(x_test)

x_train_normalized.shape

(116651, 4)

In [17]:
#create Model structure
with tf.device('/device:CPU:0'):
    model = Sequential()
    model.add(Dense(45, kernel_initializer='normal',  activation='relu', input_shape=(4,) ))    
    model.add(Dense(25, kernel_initializer='normal',activation='relu'))   
    model.add(Dense(25, kernel_initializer='normal',activation='relu'))   
    model.add(Dense(1, kernel_initializer='normal',activation=None))

In [18]:
gpus = tf.config.experimental.list_physical_devices('CPU')
print(gpus)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [19]:
#set model loss and optimizer
with tf.device('/device:CPU:0'):
    model.compile(optimizer= Adam(learning_rate=0.001),loss='mae', metrics=['mae','mape'])

In [20]:
#checkpoint to save best Model which isnt overfitted
checkpoint = ModelCheckpoint('Model1', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='min' )

In [21]:
#train Model
with tf.device('/device:CPU:0'):
    history = model.fit(x_train_normalized, y_train.values, epochs=50 , batch_size=1,shuffle=False   )

Epoch 1/50
116651/116651 [==============================] - 37s 317us/step - loss: 293.2210 - mae: 293.2210 - mape: 246.4475
Epoch 2/50
116651/116651 [==============================] - 37s 318us/step - loss: 278.7054 - mae: 278.7054 - mape: 240.0197
Epoch 3/50
116651/116651 [==============================] - 37s 319us/step - loss: 225.8078 - mae: 225.8078 - mape: 208.1079
Epoch 4/50
116651/116651 [==============================] - 37s 321us/step - loss: 206.3539 - mae: 206.3539 - mape: 205.8629
Epoch 5/50
116651/116651 [==============================] - 37s 320us/step - loss: 281.7897 - mae: 281.7897 - mape: 213.5283
Epoch 6/50
116651/116651 [==============================] - 37s 316us/step - loss: 324.6904 - mae: 324.6904 - mape: 251.9012
Epoch 7/50
116651/116651 [==============================] - 37s 316us/step - loss: 324.8920 - mae: 324.8920 - mape: 251.8401
Epoch 8/50
116651/116651 [==============================] - 37s 321us/step - loss: 324.8575 - mae: 324.8575 - mape: 251.7201s

KeyboardInterrupt: 

In [23]:
model.evaluate(x_test_normalized,y_test.values)

912/912 [==============================] - 3s 4ms/step - loss: 29163.0000 - mae: 29163.0000 - mape: 29163.0000


[29163.0, 29163.0, 29163.0]